In [1]:
import datasets 
import numpy as np 
from transformers import BertTokenizerFast 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 
from sklearn.model_selection import train_test_split
from datasets import Dataset

import warnings
warnings.filterwarnings('ignore')

# https://github.com/rohan-paul/MachineLearning-DeepLearning-Code-for-my-YouTube-Channel/blob/master/NLP/YT_Fine_tuning_BERT_NER_v1.ipynb

/Users/julianbehrendt/opt/anaconda3/envs/computational_semantics/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-31 18:07:53.302724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# This code block reads the data.txt file and outputs a list of lists with the tokens and
# a list of list of the semantic tags

full_sen = []
with open('train.it.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break    

tokens = []
tags = []
train_tags = []
train_token =[]
train_data = []
train =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            train_token.append(tokens) 
            tokens = []
            train_tags.append(tags)
            tags = []
            train_data.append(train)
            train = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            train.append((string[0],string[3]))
            
print("The training data set has",len(train_data), "sentences." )

The training data set has 684 sentences.


In [3]:
full_sen = []
with open('test.it.txt') as fh:
    # Skip initial comments that starts with #
    while True:
        line = fh.readline()
        # break while statement if it is not a comment line
        # i.e. does not startwith #
        if not line.startswith('#'):
            full_sen.append(line) 
        if not line:
            break   
tokens = []
tags = []
test_token = []
test_tags =[]
test_data = []
test =[]
for i in range (0, len(full_sen)):
    string = full_sen[i].split("\t")
    if not len(full_sen[i]) == 0: 
        if string[0] == '\n':
            test_token.append(tokens) 
            tokens = []
            test_tags.append(tags)
            tags = []
            test_data.append(train)
            test = []
        else:
            tokens.append(string[0])
            tags.append(string[3])
            test.append((string[0],string[3]))

print("The testing data set has",len(test_data), "sentences.")

The testing data set has 460 sentences.


In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

In [5]:
def transform_into_ints(data,mydict):
    for sentences in range(0,len(data)):
        sent = data[sentences]
        for i in range(0,len(sent)):
            word = sent[i]
            transformation = mydict[word]
            sent[i] = transformation
    return data

In [6]:
mydict = np.load('universal_dict.npy',allow_pickle='TRUE').item()

In [7]:
train_tags_transformed = train_tags
test_tags_transformed = test_tags

transformed_input_train = transform_into_ints(train_tags_transformed,mydict)
transformed_input_test = transform_into_ints(test_tags_transformed,mydict)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
         train_token, transformed_input_train, test_size=0.3, random_state=18)

In [9]:
from collections import Counter

D = y_train

# Flatten the nested list
flattened_list = [item for sublist in D for item in sublist]

# Count the frequencies of each value
counter = Counter(flattened_list)

# Print the frequencies of each value
print(counter)

Counter({13: 458, 1: 276, 2: 239, 5: 222, 23: 181, 12: 129, 6: 112, 9: 111, 21: 107, 0: 89, 18: 85, 29: 78, 7: 54, 3: 44, 34: 37, 26: 35, 20: 34, 19: 34, 31: 31, 39: 30, 11: 28, 4: 26, 14: 23, 24: 21, 15: 16, 30: 15, 43: 14, 16: 12, 41: 11, 22: 10, 38: 10, 17: 9, 53: 9, 37: 7, 62: 7, 35: 6, 50: 6, 25: 5, 44: 4, 49: 4, 10: 4, 42: 4, 45: 3, 28: 2, 33: 2, 32: 2, 52: 2, 58: 2, 40: 1, 48: 1, 47: 1, 59: 1, 54: 1, 8: 1, 55: 1})


In [10]:
def get_ids(tokens, tags):
    ids = []
    token = []
    ner_tags = []
    for i in range(0, len(tokens)):
        ids.append(i)
        token.append(tokens[i])
        ner_tags.append(tags[i])
    return ids, token, ner_tags

In [11]:
# To get it into the correct form: https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

ids_train, tokens_train, ner_tags_train = get_ids(X_train, y_train)
ids_val, tokens_val, ner_tags_val = get_ids(X_val, y_val)

ids_test, tokens_test, ner_tags_test = get_ids(test_token, transformed_input_test)

train = {'input_ids': ids_train,
            'tokens': tokens_train,
            'ner_tags': ner_tags_train}

validation = {'input_ids': ids_val,
            'tokens': tokens_val,
            'ner_tags': ner_tags_val}

test = {'input_ids': ids_test,
            'tokens': tokens_test,
            'ner_tags': ner_tags_test}

train = Dataset.from_dict(train) 
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)

data = {'train': train,
            'validation': validation,
            'test': test}

In [12]:
conll2003 = datasets.load_dataset("conll2003") 
conll2003

conll2003['train'] = data['train']
conll2003['validation'] = data['validation']
conll2003['test'] = data['test']

Found cached dataset conll2003 (/Users/julianbehrendt/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 214.82it/s]


In [13]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 478
    })
    validation: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 206
    })
    test: Dataset({
        features: ['input_ids', 'tokens', 'ner_tags'],
        num_rows: 460
    })
})

In [14]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 

    
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 

    
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [15]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

100%|██████████| 1/1 [00:00<00:00, 26.74ba/s]


In [16]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels= 69)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [17]:
from transformers import TrainingArguments, Trainer 
args = TrainingArguments( 
    "test-ner",
    evaluation_strategy = "epoch", 
    learning_rate=2e-5, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32, 
    num_train_epochs=10, 
    weight_decay=0.01, 
    eval_steps = 100,  
    save_total_limit = 2
) 

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer) 

In [19]:
metric = datasets.load_metric("seqeval") 

In [20]:
def compute_metrics(eval_preds): 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    
    # print(pred_logits)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    predictions = []
    true_labels = []
    for i in range(len(pred_logits)):
        pred_seq = []
        true_seq = []
        for j in range(len(pred_logits[i])):
            if labels[i][j] != -100:
                pred_seq.append(pred_logits[i][j])
                true_seq.append(labels[i][j])
        predictions.append(pred_seq)
        true_labels.append(true_seq)
    
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [21]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [22]:
trainer.train() 

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 478
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 150
  Number of trainable parameters = 108944709
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,3.084930,0.437220,0.203975,0.278174,0.290909
2,No log,2.592833,0.485666,0.301255,0.371853,0.392929
3,No log,2.258598,0.453125,0.364017,0.403712,0.437879
4,No log,2.009289,0.472959,0.466527,0.469721,0.524242
5,No log,1.809215,0.524607,0.524059,0.524333,0.578788
6,No log,1.669311,0.555556,0.570084,0.562726,0.616667
7,No log,1.584316,0.565045,0.595188,0.579725,0.640404
8,No log,1.526986,0.597737,0.607741,0.602697,0.653535
9,No log,1.477333,0.589431,0.606695,0.597938,0.663131
10,No log,1.466095,0.603640,0.624477,0.613882,0.666667


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 206
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 206
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
 

TrainOutput(global_step=150, training_loss=2.0924471028645835, metrics={'train_runtime': 603.7646, 'train_samples_per_second': 7.917, 'train_steps_per_second': 0.248, 'total_flos': 45579987400428.0, 'train_loss': 2.0924471028645835, 'epoch': 10.0})

In [23]:
model.save_pretrained("bert_it")

Configuration saved in bert_it/config.json
Model weights saved in bert_it/pytorch_model.bin


In [24]:
model_fine_tuned_italien = AutoModelForTokenClassification.from_pretrained("bert_it")

loading configuration file bert_it/config.json
Model config BertConfig {
  "_name_or_path": "bert_it",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
  

In [25]:
from transformers import pipeline
import re
len_test = 460

In [26]:
nlp = pipeline("ner", model= model_fine_tuned_italien, tokenizer=tokenizer)

all_true_labels = []
all_prediction_labels = []
for i in range(0,len_test):
    test_data = conll2003['test'][i] 
    
    true_labels = test_data['ner_tags']
    all_true_labels.append(true_labels)
    
    tokens = test_data['tokens']
    ner_predictions = nlp(tokens)
    
    prediction_labels = []
    for i in range(0, len(ner_predictions)):
        x = ner_predictions[i]
        s = x[0]
        string = s['entity']
        label = int(re.search(r'\d+', string).group())
        prediction_labels.append(label)
    
    
    all_prediction_labels.append(prediction_labels)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [27]:
all_prediction_labels

[[5, 5, 23, 5, 13],
 [5, 12, 5, 13, 2, 5, 13],
 [5, 5, 23, 5, 13],
 [5, 5, 13, 13, 18, 13, 34],
 [5, 2, 12, 23, 23, 13],
 [5, 5, 9, 5, 13],
 [14, 2, 18, 9, 13],
 [5, 13, 2, 18, 13],
 [5, 6, 12, 13],
 [0, 1, 18, 13, 0, 18, 13],
 [23, 1, 5, 2, 1, 13],
 [9, 2, 23, 5, 9, 13],
 [2, 5, 13],
 [5, 5, 6, 13, 6, 13],
 [5, 5, 2, 12, 13],
 [9, 5, 0, 12, 13],
 [5, 1, 23, 1, 13, 14, 1, 13],
 [6, 12, 13],
 [5, 5, 23, 18, 13],
 [5, 2, 18, 5, 13],
 [5, 2, 12, 12, 13],
 [14, 2, 5, 12, 13],
 [23, 6, 1, 2, 5, 13, 5, 5, 13],
 [23, 5, 13, 5, 2, 1, 13],
 [9, 5, 23, 4, 1, 13],
 [23, 1, 5, 13],
 [5, 1, 13],
 [5, 5, 23, 5, 5, 5, 13],
 [5, 6, 13],
 [5, 2, 5, 5, 13],
 [9, 13, 5, 23, 4, 5, 13],
 [5, 5, 23, 5, 13],
 [5, 6, 23, 5, 23, 5, 13],
 [5, 2, 5, 5, 13, 2, 5, 13],
 [9, 5, 13, 5, 5, 14, 1, 13],
 [23, 6, 5, 2, 6, 13],
 [9, 12, 5, 13],
 [9, 18, 13, 23, 5, 12, 13],
 [5, 18, 23, 23, 13],
 [13, 2, 6, 6, 6, 13],
 [13, 5, 5, 34],
 [5, 6, 23, 1, 13],
 [0, 2, 18, 23, 12, 13, 5, 13],
 [23, 4, 6, 13, 6, 13],
 [9, 6, 0, 5

In [28]:
all_true_labels

[[17, 18, 23, 17, 13],
 [19, 1, 20, 11, 21, 5, 13],
 [5, 18, 23, 22, 13],
 [34, 18, 23, 13, 35, 13, 34],
 [5, 2, 3, 23, 1, 13],
 [9, 18, 0, 1, 13],
 [14, 2, 15, 12, 13],
 [5, 9, 2, 29, 13],
 [5, 6, 12, 13],
 [0, 1, 6, 21, 0, 1, 13],
 [23, 1, 26, 2, 41, 13],
 [9, 2, 3, 21, 9, 13],
 [2, 12, 13],
 [5, 26, 62, 21, 0, 13],
 [5, 26, 2, 12, 13],
 [9, 6, 19, 1, 13],
 [20, 11, 0, 1, 21, 14, 1, 13],
 [20, 12, 13],
 [5, 18, 23, 1, 13],
 [5, 2, 29, 5, 13],
 [5, 2, 15, 12, 13],
 [14, 2, 25, 12, 38],
 [23, 4, 1, 2, 29, 23, 17, 32, 13],
 [23, 1, 21, 5, 2, 41, 13],
 [9, 6, 23, 4, 1, 13],
 [23, 1, 18, 13],
 [2, 12, 13],
 [5, 18, 23, 48, 47, 22, 13],
 [20, 29, 13],
 [5, 2, 3, 5, 13],
 [9, 9, 18, 23, 4, 1, 13],
 [5, 18, 23, 22, 13],
 [5, 18, 23, 1, 21, 22, 13],
 [5, 2, 25, 12, 21, 4, 7, 13],
 [9, 18, 19, 24, 21, 14, 1, 13],
 [23, 7, 5, 2, 29, 13],
 [9, 18, 1, 13],
 [9, 6, 21, 0, 1, 12, 13],
 [5, 18, 23, 1, 13],
 [13, 6, 49, 7, 37, 13],
 [34, 18, 5, 34],
 [5, 18, 23, 1, 13],
 [0, 2, 29, 23, 1, 21, 5, 13],

In [29]:
results = metric.compute(predictions=all_prediction_labels, references=all_true_labels) 


In [30]:
results

{'0': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 49},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 147},
 '2': {'precision': 0.3443708609271523,
  'recall': 0.3880597014925373,
  'f1': 0.3649122807017544,
  'number': 134},
 '3': {'precision': 0.7847593582887701,
  'recall': 0.9347133757961783,
  'f1': 0.8531976744186046,
  'number': 628},
 '4': {'precision': 0.9387755102040817,
  'recall': 0.5168539325842697,
  'f1': 0.6666666666666667,
  'number': 89},
 '5': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 31},
 '6': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 53},
 '7': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 28},
 '8': {'precision': 0.07758620689655173,
  'recall': 0.08035714285714286,
  'f1': 0.07894736842105264,
  'number': 112},
 '9': {'precision': 0.6666666666666666,
  'recall': 0.018518518518518517,
  'f1': 0.036036036036036036,
  'number': 108},
 '_': {'precision': 0.345985401459854,
  'recall': 0.34699853587115664,
 